## Pyspark installation on local machine

1. Install jdk 8 or higher via https://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html
2. install pyspark with `brew install apache-spark`
3. install pyspark and findspark via 
    - `pip install findspark`
    - `pip install pyspark`
4. In either your bashrc, zshrc, or whatever shell config you use, enter this:
    * export PYSPARK_PYTHON=python3
    * export SPARK_HOME=/usr/local/lib/python3.7/site-packages/pyspark
5. In your text editor with a python file open, type the following:

In [1]:
# from pyspark import SparkContext
# sc = SparkContext()
# from pyspark.sql import SparkSession

# spark = SparkSession \
#    .builder \
#    .appName("Python Spark regression example") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()

In [27]:
!ls

setting environment
Advertising.csv        img_35.jpg             multiTimeline.csv
Churn_Modelling.csv    multiTimeline copy.csv


In [1]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD as lrSGD

spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

regressionDataFrame = spark.read.csv('Datasets/Advertising.csv',header=True, inferSchema = True)

regressionDataFrame = regressionDataFrame.drop('_c0')

regressionDataFrame.show(10)

regressionDataRDD = regressionDataFrame.rdd.map(list)


regressionDataLabelPoint = regressionDataRDD.map(lambda data : LabeledPoint(data[3], data[0:3]))

regressionLabelPointSplit = regressionDataLabelPoint.randomSplit([0.7, 0.3])

regressionLabelPointTrainData = regressionLabelPointSplit[0]

regressionLabelPointTestData = regressionLabelPointSplit[1]


ourModelWithLinearRegression  = lrSGD.train(data = regressionLabelPointTrainData, 
                                            iterations = 200, step = 0.02, intercept = True)

+-----+-----+---------+-----+
|   TV|radio|newspaper|sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|  8.7| 48.9|     75.0|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|199.8|  2.6|     21.2| 10.6|
+-----+-----+---------+-----+
only showing top 10 rows



### We can run Pyspark in at least 3 ways: 
- 1- Local Machine (learn Pyspark syntax) 
- 2- Use AWS-SageMaker 
- 3- Use Databricks (this is the best option)

https://spark.apache.org/downloads.html

# Reasons to use Pyspark: 

- 1- Big data array manipulations
- 2- Big data-frame manipulations
- 3- Train Machine Learning Models with Big Data

# RDD 

Spark uses Resilient Distributed Datasets (RDD) to perform parallel processing across a cluster or computer processors.

In [4]:
import keras

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [5]:
from pyspark import SparkContext
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-ee18f58143ad>:2 

In [6]:
pythonList = [2.3,3.4,4.3,2.4,2.3,4.0]

In [7]:
parPythonData = sc.parallelize(pythonList,2)

# What is the type of parPythonData?

## Can we get the first element of parPythonData like parPythonData[0]?

In [8]:
parPythonData.collect()

[2.3, 3.4, 4.3, 2.4, 2.3, 4.0]

In [9]:
parPythonData.first()

2.3

In [10]:
parPythonData.take(2)

[2.3, 3.4]

In [11]:
parPythonData.getNumPartitions()    # 2,we passed two when we initialized parPythonData earlier

2

In [12]:
tempData = [59,57.2,53.6,55.4,51.8,53.6,55.4]

In [13]:
parTempData = sc.parallelize(tempData, 2)

In [14]:
parTempData.collect()

[59, 57.2, 53.6, 55.4, 51.8, 53.6, 55.4]

In [15]:
def get_fahrenheitToCentrigrade(temperature):
    centrigrade = (temperature-32)*5/9
    return centrigrade

In [16]:
get_fahrenheitToCentrigrade(59)

15.0

# Compare map in Python and Pyspark

In [17]:
list(map(get_fahrenheitToCentrigrade, tempData))

[15.0, 14.000000000000002, 12.0, 13.0, 10.999999999999998, 12.0, 13.0]

In [18]:
parCentrigradeData = parTempData.map(get_fahrenheitToCentrigrade)

In [19]:
parCentrigradeData.collect()

[15.0, 14.000000000000002, 12.0, 13.0, 10.999999999999998, 12.0, 13.0]

In [20]:
def tempMoreThanThirteen(temperature):
    return tempurature >= 13


In [21]:
filteredTemperature = parCentrigradeData.filter(tempMoreThanThirteen)

In [22]:
filteredTemperature.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 8, 192.168.1.12, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/alannanoguchi/opt/miniconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/Users/alannanoguchi/opt/miniconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Users/alannanoguchi/opt/miniconda3/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/alannanoguchi/opt/miniconda3/lib/python3.8/site-packages/pysp

In [23]:
filteredTemperature = parCentrigradeData.filter(lambda x: x >= 13)

In [24]:
filteredTemperature.collect()

[15.0, 14.000000000000002, 13.0, 13.0]

# Work with text data

In [32]:
lines = sc.textFile("data.txt")
lineLengths = lines.map(lambda s: len(s))
totalLengths = lineLengths.reduce(lambda a, b: a + b)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/alannanoguchi/dev/DS/DS2.3/DS-2.3-Data-Science-in-Production/Class Notes/data.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partiti

### Create data.txt with 5 lines

In [33]:
rdd = sc.textFile("for_pyspark.txt")
words = rdd.flatMap(lambda x: x.split(" "))
result = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

Py4JJavaError: An error occurred while calling o222.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/alannanoguchi/dev/DS/DS2.3/DS-2.3-Data-Science-in-Production/Class Notes/for_pyspark.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache

In [34]:
result.collect()

NameError: name 'result' is not defined

In [35]:
result

NameError: name 'result' is not defined

In [36]:
words.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/alannanoguchi/dev/DS/DS2.3/DS-2.3-Data-Science-in-Production/Class Notes/for_pyspark.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.

In [37]:
words.map(lambda x: (x,1)).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/alannanoguchi/dev/DS/DS2.3/DS-2.3-Data-Science-in-Production/Class Notes/for_pyspark.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:205)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:272)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:276)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.

In [41]:
import math
# from pyspark import SparkContext

# sc = SparkContext()

nums = sc.parallelize(range(100000), numSlices=100)
doubled = nums.map(lambda n: n*2)
total = doubled.filter(lambda n: n%4==0).reduce(lambda a, b: a+b)
print(math.sqrt(total))

70709.97100833799


# Transform the above computation into a Python Function (not in map/reduce way )

In [33]:
def f(ls):
    s = 0
    for i in ls:   # for item in our list
        if (i*2)%4 == 0:    # 
            s += (i*2)
    return math.sqrt(s)
    
    
print(f(range(100000)))

70709.97100833799


### Difference Between map and flatmap

In [42]:
values = sc.parallelize([1, 2, 3, 4], 2)
print(values.map(range).collect())
# [[0], [0, 1], [0, 1, 2], [0, 1, 2, 3]]
print(values.flatMap(range).collect())
# [0, 0, 1, 0, 1, 2, 0, 1, 2, 3]

[range(0, 1), range(0, 2), range(0, 3), range(0, 4)]
[0, 0, 1, 0, 1, 2, 0, 1, 2, 3]


## Datafame in Pyspark

DataFrame is collection of named columns.

In [38]:
from pyspark import SparkContext
sc = SparkContext()

ImportError: cannot import name 'SparkContext' from 'pyspark' (/Users/alannanoguchi/dev/DS/DS2.3/DS-2.3-Data-Science-in-Production/Class Notes/pyspark.py)

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/lib/python3.7/site-packages/pyspark/./bin/spark-submit'

In [52]:
regressionDataFrame = spark.read.csv('Datasets/Advertising.csv',header=True, inferSchema = True)

In [53]:
regressionDataFrame.show(5)

+---+-----+-----+---------+-----+
|_c0|   TV|radio|newspaper|sales|
+---+-----+-----+---------+-----+
|  1|230.1| 37.8|     69.2| 22.1|
|  2| 44.5| 39.3|     45.1| 10.4|
|  3| 17.2| 45.9|     69.3|  9.3|
|  4|151.5| 41.3|     58.5| 18.5|
|  5|180.8| 10.8|     58.4| 12.9|
+---+-----+-----+---------+-----+
only showing top 5 rows



In [54]:
regressionDataFrame = regressionDataFrame.drop('_c0')

In [55]:
regressionDataFrame.show(5)

+-----+-----+---------+-----+
|   TV|radio|newspaper|sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows



In [56]:
regressionDataFrame.columns

['TV', 'radio', 'newspaper', 'sales']

In [57]:
regressionDataFrame.groupBy(regressionDataFrame.TV > 100).count().show(5)

+----------+-----+
|(TV > 100)|count|
+----------+-----+
|      true|  130|
|     false|   70|
+----------+-----+



In [58]:
regressionDataFrame.filter(regressionDataFrame.TV > 100).show(5)

+-----+-----+---------+-----+
|   TV|radio|newspaper|sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|120.2| 19.6|     11.6| 13.2|
|199.8|  2.6|     21.2| 10.6|
+-----+-----+---------+-----+
only showing top 5 rows



In [59]:
regressionDataFrame.select(regressionDataFrame.TV > 100).show(5)

+----------+
|(TV > 100)|
+----------+
|      true|
|     false|
|     false|
|      true|
|      true|
+----------+
only showing top 5 rows



In [60]:
regressionDataFrame.describe

<bound method DataFrame.describe of DataFrame[TV: double, radio: double, newspaper: double, sales: double]>

In [61]:
from pyspark.sql.functions import mean, min, max

regressionDataFrame.select([mean('TV'), min('TV'), max('TV')]).show()

+--------+-------+-------+
| avg(TV)|min(TV)|max(TV)|
+--------+-------+-------+
|147.0425|    0.7|  296.4|
+--------+-------+-------+



In [62]:
regressionDataFrame.crosstab('TV', 'radio').show()

+--------+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|TV_radio|0.0|0.3|0.4|0.8|1.3|1.4|1.5|1.6|1.9|10.0|10.1|10.6|10.8|11.0|11.6|11.7|11.8|12.0|12.1|12.6|13.9|14.0|14.3|14.5|14.7|14.8|15.4|15.5|15.8|15.9|16.0|16.7|16.9|17.0|17.2|17.4|18.1|18.4|1